This notebook demonstrates the process of adding custom layers to the CoreML model during conversion.  
CoreML supports a certain set of Neural Network layers. The list can be found [here](https://github.com/apple/coremltools/blob/master/mlmodel/format/NeuralNetwork.proto) or [here](https://apple.github.io/coremltools/coremlspecification/sections/NeuralNetwork.html).
For TensorFlow operations (ops for short) that are not translatable to any of these CoreML layers, custom layers can be inserted in the CoreML model. At runtime, CoreML framework will look for the implementation code of the custom layer, which has to be provided by the developer in her app.   
Custom layer is a [proto message](https://github.com/apple/coremltools/blob/5b5b8190764ffe78110be6b4d0edbeebe0253a6e/mlmodel/format/NeuralNetwork.proto#L2280), like any other neural netowrk layer in the .mlmodel file (which is in protobuf format), that can hold the parameters and weights (if any) associated with the TF op.
Here is more [documentation](https://developer.apple.com/documentation/coreml/core_ml_api/creating_a_custom_layer) on CoreML custom layers. 


Lets start by defining a toy TF graph. 

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.tools.freeze_graph import freeze_graph
import numpy as np
import shutil
import tempfile
import os
import tfcoreml

/Users/aseem/Aseem_env/env/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model_dir = tempfile.mkdtemp()
graph_def_file = os.path.join(model_dir, 'tf_graph.pbtxt')
checkpoint_file = os.path.join(model_dir, 'tf_model.ckpt')

graph = tf.Graph()
with graph.as_default() as g:
    inputs = tf.placeholder(tf.float32, shape=[None,8], name='input')
    with slim.arg_scope([slim.fully_connected],
          weights_initializer=tf.truncated_normal_initializer(0.0, 0.2),
          weights_regularizer=slim.l2_regularizer(0.0005)):
        y = slim.fully_connected(inputs, 10, scope='fc')
        y = slim.unit_norm(y,dim=1)
    saver = tf.train.Saver()

output_name = y.op.name
X = np.random.rand(1,8)

with tf.Session(graph = graph) as sess:
    sess.run(tf.global_variables_initializer())
    out = sess.run(y,feed_dict={inputs: X}) 
    tf.train.write_graph(sess.graph, model_dir, graph_def_file)
    saver.save(sess, checkpoint_file)
    

print 'TF out: ', output_name, out.shape, np.sum(out ** 2)

TF out:  UnitNorm/div (1, 10) 0.99999654


In [3]:
# generate the frozen .pb file
def _simple_freeze(input_graph, input_checkpoint, output_graph, output_node_names):
    freeze_graph(input_graph=input_graph,
                 input_saver="",
                 input_binary=False,
                 input_checkpoint=input_checkpoint,
                 output_node_names=output_node_names,
                 restore_op_name="save/restore_all",
                 filename_tensor_name="save/Const:0",
                 output_graph=output_graph,
                 clear_devices=True,
                 initializer_nodes="")
    

frozen_model_file = 'unit_norm_graph.pb'
_simple_freeze(input_graph=graph_def_file,
                input_checkpoint=checkpoint_file,
                output_graph=frozen_model_file,
                output_node_names=output_name)
      
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)

INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpig1YdI/tf_model.ckpt


INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpig1YdI/tf_model.ckpt


INFO:tensorflow:Froze 2 variables.


INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.


In [ ]:
import netron
netron.serve_file(frozen_model_file, browse = True, port=8090)

In [ ]:
# Try to convert it 
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path='unit_norm_graph.mlmodel',
        input_name_shape_dict={'input:0':[1,8]},
        output_feature_names=[output_name+':0'])

In [4]:
# we get an unsupported op error. Try again with custom Flag set to true
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path='unit_norm_graph.mlmodel',
        input_name_shape_dict={'input:0':[1,8]},
        output_feature_names=['UnitNorm/div:0'],
        add_custom_layers=True)

Shapes not found for 10 tensors. Executing graph to determine shapes. 
1/26: Analysing op name: UnitNorm/concat/axis ( type:  Const )
2/26: Analysing op name: UnitNorm/StridedSlice/end ( type:  Const )
3/26: Analysing op name: UnitNorm/StridedSlice/begin ( type:  Const )
4/26: Analysing op name: UnitNorm/ones_like/Const ( type:  Const )
5/26: Analysing op name: UnitNorm/ones_like/Shape ( type:  Const )
6/26: Analysing op name: UnitNorm/ones_like ( type:  Fill )
7/26: Analysing op name: UnitNorm/ones ( type:  Const )
8/26: Analysing op name: UnitNorm/add/x ( type:  Const )
9/26: Analysing op name: UnitNorm/Sum/reduction_indices ( type:  Const )
10/26: Analysing op name: fc/biases ( type:  Const )
11/26: Analysing op name: fc/biases/read ( type:  Identity )
12/26: Analysing op name: fc/weights ( type:  Const )
13/26: Analysing op name: fc/weights/read ( type:  Identity )
14/26: Analysing op name: input ( type:  Placeholder )
Skipping name of placeholder
15/26: Analysing op name: fc/MatMu

In [5]:
# visualize CoreML model
import netron
netron.serve_file('unit_norm_graph.mlmodel', browse = True, port=8091)

Reading 'unit_norm_graph.mlmodel'
Serving 'unit_norm_graph.mlmodel' at http://localhost:8091

Stopping


In [10]:
# inspect the CoreML model
import coremltools
spec = coreml_model.get_spec()
nn_layers = coremltools.models.utils._get_nn_layers(spec)
for i, layer in enumerate(nn_layers):
    print('{}: layer type: ({}) , inputs: {}, outputs: {}'.
          format(i,layer.WhichOneof('layer'), ", ".join([x for x in layer.input]), ", ".join([x for x in layer.output])))

0: layer type: (innerProduct) , inputs: input__0, outputs: fc/BiasAdd:0
1: layer type: (activation) , inputs: fc/BiasAdd:0, outputs: fc/Relu:0
2: layer type: (loadConstant) , inputs: , outputs: UnitNorm/StridedSlice:0
3: layer type: (loadConstant) , inputs: , outputs: UnitNorm/ones:0
4: layer type: (concat) , inputs: UnitNorm/ones:0, UnitNorm/StridedSlice:0, outputs: UnitNorm/concat:0
5: layer type: (multiply) , inputs: fc/Relu:0, fc/Relu:0, outputs: UnitNorm/Square:0
6: layer type: (reduce) , inputs: UnitNorm/Square:0, outputs: UnitNorm/Sum:0
7: layer type: (add) , inputs: UnitNorm/Sum:0, outputs: UnitNorm/add:0
8: layer type: (unary) , inputs: UnitNorm/add:0, outputs: UnitNorm/Sqrt:0
9: layer type: (custom) , inputs: UnitNorm/Sqrt:0, UnitNorm/concat:0, outputs: UnitNorm/Tile:0
10: layer type: (unary) , inputs: UnitNorm/Tile:0, outputs: inversed_UnitNorm/Tile:0_UnitNorm/div:0
11: layer type: (multiply) , inputs: fc/Relu:0, inversed_UnitNorm/Tile:0_UnitNorm/div:0, outputs: UnitNorm__di

In [12]:
# we see that the 10-th layer is the custom layer. Lets print it fully.
print(nn_layers[9])

name: "UnitNorm__Tile"
input: "UnitNorm/Sqrt:0"
input: "UnitNorm/concat:0"
output: "UnitNorm/Tile:0"
custom {
  className: "Tile"
  description: "Custom layer that corresponds to the TensorFlow op Tile"
}

